# Web Scraping and Data Cleansing

## Contents

- [Data Dictionary](#0)
- [Web Scraping](#1)
	- [Part 1a: Scraping from the browsing pages](#2)
	- [Part 1b: Cleaning the HTML](#3)
	- [Part 2a: Scraping from individual record pages](#4)
	- [part 2b: Cleaning the HTML](#5)

<a id="0"></a>
## Data Dictionary

- ppl_have: People in discogs community who own the record 
- ppl_want: People in community who want the record
- weighted ratio: square root( (people who want record)^2/ people who have record )
- rated: average star rating (1-5 system)
- genre: The main category/s a record falls under. (just 1 value for most records)
- style: Subcategory of genre, defining the type of music in more detail. (most records have a list of styles)
- media_condition: the condition which the vinyl is in
- sleeve_condition: the condition of the case
- age: (2019 - the year of production)
- track_no: number of tracks on the record
- format: the type of record (album, 12", LP, Single etc)
- 12": a format of vinyl that has thicker grooves for higher sound quality, but fits less music
- ships_from: country the record is selling from
- country_origin: country the record was produced in
- has_b_side: binary response for music on both sides of the record. (it's also possible that the listing contains more than 1 album)

In [ ]:
#cleansing and data handling
import pandas as pd
import numpy as np
import regex as re
import ast as ast
from sklearn.preprocessing import MultiLabelBinarizer

#web scraping
import time
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

<a id="1"></a>
# Web Scraping

<a id="2"></a>
## Part 1: Scraping the browsing pages

In [ ]:
# Scraping 400 pages of 250 records per page. Aiming to arrive with 100,000 records, though many will be duplicates
# or have missing values

search_urls=[]

for i in range(1,400):
    search_urls.append('https://www.website.com/sell/list?sort=listed%2Cdesc&limit=250&format=Vinyl&page={}'.format(i))
search_urls

In [ ]:
soup_thumb=[]     #Section of web page containing; Ratings, Users have, Users want
soup_details=[]   #Section of web page containing; Name, Sleeve, Media condition, (Record URLs contained within this block of HTML)
soup_seller=[]    #Section of web page containing; Seller, Rating, Ratings, Ships From
soup_money=[]     #Section of web page containing; price


#  Using Selenium to automate a google chrome session, which will loop through webpages, and pull all of the HTML code

#  Then using Beautiful soup to process the HTML. Identifying which interval in the webpage is a new record 
#  and dividing the HTML into sections for each record, making for easier processing later on.

driver = webdriver.Chrome('/Users/josephstern/Downloads/chromedriver')
for i in tqdm(search_urls):

    driver.get(i)
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    for info in soup.find_all('td', attrs={'class':'item_picture as_float'}):
        try:
            soup_thumb.append(info)
        except:
            soup_thumb.append(np.nan)
            
    for info in soup.find_all('td', attrs={'class':'item_description'}):
        try:
            soup_details.append(info)
        except:
            soup_details.append(np.nan)
    
    for info in soup.find_all('td', attrs={'class':'seller_info'}):
        try:
            soup_seller.append(info)
        except:
            soup_seller.append(np.nan)
            
    for info in soup.find_all('td', attrs={'class':'item_price hide_mobile'}):
        try:
            soup_money.append(info.find('span', attrs={'class':'price'}).text.strip())
        except:
            soup_money.append(np.nan)
            
driver.quit()

In [ ]:
# Using brute force methods to locate the data within each block of HTML.
# despite the absolute chaos within the HTML, it is consistent for almost every record

#from soup thumb
rated=[]
have=[]
want=[]

for thing in soup_thumb:
    if 'Rated:' in thing.text:
        rated.append(float(re.search(r"Rated: (.*)\n\n\n\n\n",thing.text).group(1)))
    else:
        rated.append(np.nan)
        
    if 'have' in thing.text:
        have.append(int(re.search(r"\n(.*)have",thing.text).group(1)))
    else:
        have.append(np.nan)

    if 'want' in thing.text:
        want.append(int(re.search(r"\n(.*)want",thing.text).group(1)))
    else:
        want.append(np.nan)

#from soup details        
name=[]
media_cond=[]
sleeve_cond=[]
label=[]

for thing in soup_details:
    if '\n' in thing.text:
        name.append(str(re.search(r"\n\n(.*)\n\n\n",thing.text).group(1)))
    else:
        name.append(np.nan)
        
    if 'Label:' in thing.text:
        label.append(str(re.search(r"Label:(.*)\n",thing.text).group(1)))
    else:
        label.append(np.nan)
        
    if 'Media:' in thing.text:
        media_cond.append(str(re.search(r"Media:\n\n(.*)\n\n",thing.text).group(1).strip()))
    else:
        media_cond.append(np.nan)

    if 'Sleeve:' in thing.text:
        sleeve_cond.append(str(re.search(r"Sleeve:\n(.*)\n\n\n\n",thing.text).group(1).strip()))
    else:
        sleeve_cond.append(np.nan)

#from soup seller
seller=[]
av_rating=[]
ratings=[]
ships_from=[]

for thing in soup_seller:
    if 'Seller:' in thing.text:
        seller.append(str(re.search(r"Seller:\n(.*)\n",thing.text).group(1)))
    else:
        seller.append(np.nan)
        
    if '%' in thing.text:
        av_rating.append(float(re.search(r"\n\n\n\n(.*)%",thing.text).group(1).strip()))
    else:
        av_rating.append(np.nan)
        
    if 'ratings' in thing.text:
        ratings.append(int(re.search(r"\n(.*)ratings",thing.text).group(1).strip().replace(',','')))
    else:
        ratings.append(np.nan)
        
    if 'Ships From:' in thing.text:
        ships_from.append(str(re.search(r"From:(.*)\n\n\n\n\n",thing.text).group(1).strip()))
    else:
        ships_from.append(np.nan)

#checks
check_list=[ratings, have, want, name, label, media_cond, sleeve_cond, seller, av_rating, ratings, ships_from]

for item in check_list:
    print(len(item))

<a id="3"></a>
### Cleaning Part 1

In [ ]:
#removing unnecessary characters and converting currencies

#name
artist=[re.sub("[\(\[].*?[\)\]]", "", x).replace('*','').split(' - ')[0] for x in name]
album=[re.sub("[\(\[].*?[\)\]]", "", x).replace('*','').split(' - ')[1] for x in name]

#label
label_=[re.sub("[\(\[].*?[\)\]]", "", x) for x in label]

#price
currency_3d=['CA$','CHF','SEK','NZ$','MX$','ZAR']
currency_2d=['A$','R$']

gbp=[]
currency=[]
value=[]

for price in soup_money:
    try:
        if price[:3] in currency_3d:
            currency.append(price[:3])
            value.append(float(price[3:].replace(',','.')))
        elif price[:2] in currency_2d:
            currency.append(price[:2])
            value.append(float(price[2:].replace(',','.')))                
        else:
            currency.append(price[0])
            value.append(float(price[1:].replace(',','.')))
    except:
        value.append(np.nan)

for price in list(zip(currency,value)):
    if price[0]=='$':
        gbp.append(round(price[1]*0.79,2))
    elif price[0]=='€':
        gbp.append(round(price[1]*0.9,2))
    elif price[0]=='CA$':
        gbp.append(round(price[1]*0.58,2))
    elif price[0]=='A$':
        gbp.append(round(price[1]*0.56,2))
    elif price[0]=='CHF':
        gbp.append(round(price[1]*0.8,2))
    elif price[0]=='¥':
        gbp.append(round(price[1]*0.0071,2))
    elif price[0]=='SEK':
        gbp.append(round(price[1]*0.088,2)) 
    elif price[0]=='R$':
        gbp.append(round(price[1]*0.21,2))
    elif price[0]=='NZ$':
        gbp.append(round(price[1]*0.53,2)) 
    elif price[0]=='MX$':
        gbp.append(round(price[1]*0.04,2))
    elif price[0]=='ZAR':
        gbp.append(round(price[1]*0.55,2)) 
    elif price[0]=='£':
        gbp.append(price[1])
    else:
        gbp.append(np.nan)

In [ ]:
# URL list for 2nd part of the web scraping. essentially creating a list of 100,000 web pages to scrape at individual
# record level
url_links=[]
for i in range(len(soup_details)):
    url_links.append('https://www.website.com'+str(soup_details[i].find_all('a')[0]).split(' ')[3].split('"')[1])
    
print(len(url_links))

In [ ]:
#putting first round of scraping results into an organised dataframe
df1=pd.DataFrame({'currency':currency,
                  'pound_value':gbp,
                  'artist':artist,
                  'album':album,
                  'rated':rated,
                  'ppl_have':have,
                 'ppl_want':want,
                 'wh_ratio':ratio,
                 'weighted_ratio':ratio_w,
                 'vinyl_condition':media_cond,
                 'sleeve_condition':sleeve_cond,
                 'label':label,
                 'seller':seller,
                 'seller_rating':av_rating,
                 'rated_sales':ratings,
                 'ships_from':ships_from,
                 'url':url_links
                 })

#test=df1.artist+df1.album
#print('duplicates:',1-len(pd.DataFrame(test)[0].unique())/len(df1))

#Dropping duplicate records
df1=df1.drop_duplicates(subset=['artist', 'album'], keep='first')

#Dealing with Missing Values
df1['ppl_have']=df1['ppl_have'].fillna(0)
df1['ppl_want']=df1['ppl_want'].fillna(0)
df1['weighted_ratio']=df1['weighted_ratio'].fillna(0)
df1['sleeve_condition']=df1['sleeve_condition'].fillna('Very Good Plus (VG+)')  
df1['seller_rating']=df1['seller_rating'].fillna(df1.seller_rating.mean())
df1['rated_sales']=df1['rated_sales'].fillna(0)

#Just dropping records with no rating. Bad idea to fill with mean as it is quite influential later on
df1=df1.dropna()

#Exporting to CSV
df1.to_csv('my path/df1.csv')

<a id="4"></a>
### Part 2: Scraping individual record pages

In [ ]:
df1=pd.read_csv('my path/df1.csv')

In [ ]:
soup_details2=[]
soup_tracks=[]
url=[]    #will be needed for inner joining, especially if the lengths don't add up
x=0   #I put this in place as a sort of bookmark, so i could scrape ~10,000 records at a time 
      #and check my results before proceeding

In [ ]:
# Creating a bot to interrogate each page individually. This is the long part. 
# scraping 8 pages at a time in warp speed then sleeping for 5 seconds.
# Most efficient way I could find that avoids being blocked by website.

driver = webdriver.Chrome('/Users/josephstern/Downloads/chromedriver')
#driver = webdriver.Firefox(executable_path=r'/Users/josephstern/Downloads/geckodriver') #Firefox was slower

for j in tqdm(range(12500)):
    time.sleep(5)
    for i in df1.url[x:(x+8)]:
        driver.get(i)
        html = driver.page_source
        soup = BeautifulSoup(html)

        for info in soup.find_all('div', attrs={'class':'profile'}):
            try:
                soup_details2.append(info)
                url.append(i)
            except:
                soup_details2.append(np.nan)
                url.append(i)

        for tracks in soup.find_all('table', attrs={'class':'playlist'}):
            try:
                soup_tracks.append(tracks)
            except:
                soup_tracks.append(np.nan)
        x+=1
        
driver.quit()

#checking how many pages were browsed and that the lists add up
x,len(soup_details2),len(soup_tracks),len(url)

In [ ]:
#Same steps as before, extracting details that can only be seen on the individual record pages. 

format_=[]
country=[]
year=[]
genre=[]
style=[]

#Scraped from the main block of details
for thing in soup_details2:

    if 'Format:' in thing.text:
        format_.append(re.search(r"Format:(.*)\n",thing.text.replace(' ','').replace('\n\n',' ')).group(1).strip())
    else:
        format_.append(np.nan)

    if 'Country:' in thing.text:
        country.append(re.search(r"Country:(.*)\n",thing.text.replace('\n\n',' ')).group(1).strip())
    else:
        country.append(np.nan)

    if 'Released:' in thing.text:
        year.append(re.search(r"Released:(.*)\n",thing.text.replace('\n\n',' ')).group(1).strip())
    else:
        year.append(np.nan)

    if 'Genre:' in thing.text:
        try:
            genre.append(re.search(r"Genre: \n(.*)\n",thing.text.replace(' ','').replace('\n\n',' ')).group(1).strip())
        except:
            genre.append(np.nan)

    if 'Style:' in thing.text:
        style.append(thing.text.replace(' ','').replace('\n\n',' ').split('\n')[-1].strip())
    else:
        style.append(np.nan)

        
#Some tracks are split by 6 break spaces, some are split by 3.      

tracks=[]
for thing in soup_tracks:
    try:
        if '\n\n\n\n\n\n' in thing.text:
            tracks.append(len(thing.text.split('\n\n\n\n\n\n'))-1)
        elif '\n\n\n' in thing.text:
            tracks.append(len(thing.text.split('\n\n\n'))-1)
        else:
            tracks.append(np.nan)
    except:
        tracks.append(np.nan)
        
# Creating a new feature to indicate the presence of a B side.

b_side=[]
for x in soup_tracks:
    if 'B1' in x.text:
        b_side.append(1)
    else:
        b_side.append(0)
        

#checks
check_list=[format_, country, year, genre, style, tracks]

for item in check_list:
    print(len(item))   


<a id="5"></a>
### Cleaning Part 2

In [ ]:
#DATA CLEANING

#country
country_=[np.nan if x=='' else x for x in country]

#year
year_=[]
for x in year:
    try:
        if x=='':
            year_.append(np.nan)
        else:
            year_.append(int(x[-4:]))
    except:
        year_.append(np.nan)


#style
style_=[np.nan if x=='Style:' else x.split(',') for x in style]

#genre
genre_=[]
for x in genre:
    try:
        if x=='':
            genre_.append(np.nan)
        else:
            genre_.append(x.split(','))
    except:
        genre_.append(np.nan)


#format
format_2=[x.split(',') for x in format_]

In [ ]:
df2=pd.DataFrame({'format_':format_2,
              'country_origin':country_,
              'year':year_,
              'genre':genre_,
              'style':style_,
              'has_b_side':b_side,
              'track_no':tracks,
              'url':url})

df2.to_csv('my path /df2.csv')

In [ ]:
df3=pd.merge(df1,df2,on='url',how='inner')

### Cleansing Final Datasets

In [ ]:
#getting rid of un-named columns columns
df4=pd.concat([df3.iloc[:,2:18], df3.iloc[:,19:26]], axis=1, sort=False)

#dropping records with track counts that look wrong- could be mistake as the cleansing criteria was quite loose, 
    #but also could be massive compilations or something weird 
df5=df4[df4.track_no<24]

#filling blank year with average per genre
df5['year'] = df5.groupby(['genre'])['year'].transform(lambda x: x.fillna(x.mean()))

#dropping all other values
df6=df5.dropna()

In [ ]:
# Creating a new binarized dataset, for Format, Genre and Style columns.
# these columns are lists of lists, so i needed to do something else with them
# keeping this seperate from main dataset for now as it'll add ~500 extra columns

#stored as objects for some reason, converting back to list.
test_format=[ast.literal_eval(x) for x in df6.format_]
test_genre=[ast.literal_eval(x) for x in df6.genre]
test_style=[ast.literal_eval(x) for x in df6['style']]

#FORMATS
df = pd.DataFrame({'groups':test_format}, columns=['groups'])
s = df['groups']
mlb = MultiLabelBinarizer()
formats=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

#just keeping the most common formats as there are way too many here
df6.format_.value_counts()

acceptable_formats=['Vinyl','LP','Album','12"','45RPM','33⅓RPM',
                '7"','Single','2×','Compilation','EP','Reissue',
                'Stereo','Promo','Mono','Gatefold','Maxi-Single','3×']

formats=formats[acceptable_formats]


#GENRES
df = pd.DataFrame({'groups':test_genre}, columns=['groups'])
s = df['groups']
mlb = MultiLabelBinarizer()
genres=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)
genre_list=list(genres.columns)

#STYLES
df = pd.DataFrame({'groups':test_style}, columns=['groups'])
s = df['groups']
mlb = MultiLabelBinarizer()
styles=pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)



In [ ]:
binary_cols=pd.concat([formats,genres,styles], axis=1, sort=False)
binary_cols.to_csv('my path/binary_cols.csv')

In [ ]:
#keeping the lists of lists out of my exploratory analysis for now
df7=df6.drop(columns=['format_','style','genre'])
df7.columns
df7.to_csv('/Users/josephstern/Desktop/project_files/df7.csv')